# Macro

In [ ]:
# Widgets in this script shall be exec under Python3.7
# Unexpected error happens for 3.10

%matplotlib widget
# %matplotlib notebook

import ipywidgets
from IPython.display import display, clear_output
# import IPython.display as display

from scipy.integrate import odeint

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['text.usetex'] = True

# Motion equation in space colony frame

In [ ]:
def motion_colony_frame(delta, t, omega):
    # Effective motion equation in Space Colony frame
    r, theta, Dr, Dtheta = delta
    # Dr, Dtheta, DDr, DDtheta, respectively
    Ddelta = np.array([
        Dr, 
        Dtheta, 
        2*omega**2*r+2*omega*Dtheta, 
        -omega**2*theta-2*omega*Dr
        ])
    return Ddelta


def sim_Track_Colony(
    v1 = 1.0,
    tMax = 10.0,
    omega = 1.0,
    c = 1.0,
    plot_display = True
):
    # Simulator of motion in Space Colony frame
    tSeries_Colony = np.linspace(
        # Time series
        0,
        tMax/omega,
        1000
    )
    init_Colony = (
        # r0, theta0, Dr0, Dtheta0, respectively
        c,
        0.0,
        0.0,
        -c*omega*v1
    )

    # Track simulated
    track_Colony = odeint(
        motion_colony_frame,
        init_Colony,
        tSeries_Colony,
        args=(
            omega,
        )
    )
    X, Y = track_Colony[:, 0], track_Colony[:, 1]

    # Plot
    # plt.clf()
    # plt.close('all')
    fig_Colony, ax_Colony = plt.subplots(
        1, 1
    )
    ax_Colony.plot(X, Y, label='Colony')
    ax_Colony.legend()

    # if plot_display==False:
    #     fig_Colony.clf()

    return fig_Colony

# Interactive

In [ ]:
# Control widgets, layout / ToggleButton
out = ipywidgets.Output()

layout_row = ipywidgets.Layout(
    display='flex',
    flex_flow = 'row',
    align_items ='center',
    justify_content = 'center'
)
layout_column = ipywidgets.Layout(
    display='flex',
    flex_flow = 'column',
    align_items ='center',
    justify_content = 'center'
)

button_display = ipywidgets.ToggleButton(
    value = True,
    description='Sim',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Simulate',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
box_display = ipywidgets.VBox(
    children=[button_display],
    layout = layout_column
)
# display(box_display)

toggle_control = ipywidgets.ToggleButtons(
    options=[
        'Set',
        'Default',
        'Hide',
        'Sim'
    ],
    tooltips=[
        'Set paras',
        'Reset default paras',
        'Hide paras control',
        'Run simulation'
    ],
    value = 'Default',
    description='Control',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     icons=['check'] * 2
)
box_control = ipywidgets.HBox(
    children=[toggle_control],
    layout = layout_row
)
# display(box_control)


# Para widgets, sliders
slider_v1 = ipywidgets.FloatSlider(
    value=2.75,
    min=0.0,
    max=10.0,
    step=0.01,
    description = 'V1'
)
slider_tMax = ipywidgets.FloatSlider(
    value = 5.0,
    min=0.0,
    max=10.0,
    step=0.01,
    description = 'Time simed'
)
slider_omega = ipywidgets.FloatSlider(
    value = 1.0,
    min=0.0,
    max=10.0,
    step=0.01,
    description = 'omega'
)
slider_c = ipywidgets.FloatSlider(
    value = 1.0,
    min=0.0,
    max=10.0,
    step=0.01,
    description = 'c'
)
sliders_paras = [slider_v1, slider_tMax, slider_omega, slider_c]
box_sliders = ipywidgets.VBox(
    children=sliders_paras,
    layout = layout_column
)
accordion_paras = ipywidgets.Accordion(
    children=[slider_v1, slider_tMax, slider_omega, slider_c],
    titles = ('v1', 'tMax', 'omega', 'c')
)
# display(box_sliders)
# display(accordion_paras)


# Display widgets, interactive func / display box
plot_track_colony = ipywidgets.interactive_output(
    sim_Track_Colony,
    {
        'v1': slider_v1,
        'tMax': slider_tMax,
        'omega': slider_omega,
        'c': slider_c,
        'plot_display': button_display
    }
)
box_plot = ipywidgets.VBox(
    children=[plot_track_colony],
    layout = layout_column
)
# display(box_plot)

# Assemble together
panel_Track_Colony = ipywidgets.VBox(
    [
    box_control,
    box_sliders,
    # box_plot,
    plot_track_colony,
    ]
)

def handle_toggle_click(change):
    # 
    if change['new']=='Set':
        panel_Track_Colony.children=[
            box_control,
            box_sliders
        ]
    elif change['new']=='Default':
        panel_Track_Colony.children=[
            box_control,
            box_sliders
        ]
    elif change['new']=='Hide':
        panel_Track_Colony.children=[
            box_control
        ]
    elif change['new']=='Sim':
        # plt.clf()
        panel_Track_Colony.children=[
            box_control,
            box_sliders,
            # box_plot,
            plot_track_colony,
        ]

toggle_control.observe(handle_toggle_click, 'value')

# with out:
#     clear_output(wait=True)
display(panel_Track_Colony)



# Reclaim

In [ ]:
# track_Colony_manual = ipywidgets.interactive(
#     sim_Track_Colony,
#     {'manual':True, 'manual_name':'Simulate'},
#     v1 = slider_v1,
#     tMax = slider_tMax,
#     omega = slider_omega,
#     c = slider_c
# )
# display(track_Colony_manual.children)
# display(track_Colony_manual.result)
# track_Colony_slider = ipywidgets.VBox([
#     ipywidgets.VBox([
#         slider_v1,
#         slider_tMax,
#         slider_omega,
#         slider_c,
#         button_display
#     ]),
#     plot_track_colony
# ])
# out = ipywidgets.Output()
# with out:
#     display.clear_output()
# track_Colony_slider
# plot_track_colony
# display(track_Colony_slider)
# display(track_Colony_manual)